 # Программирование на языках Python и SQL
## Контрольные работы

Контрольные работы проводятся согласно расписанию учебных занятий в подгруппах №№ 2, 3, 4 в IV семестре. В подгруппу 2 входят студенты групп ПИ19-2, ПИ19-3. В подгруппу 3 входят студенты ПИ19-3, ПИ19-4. В подгруппу 4 входят студенты ПИ19-4, ПИ19-5.

Задания контрольных работ подготовил: Смирнов Михаил Викторович, доцент депратамента анализа данных и машинного обучения Финансового унивеситета 

Москва - 2021

## Контрольная работа № 1
<Table>
    <tr>
        <th>Подгруппа</th><th>Дата</th></tr>
    <tr>
        <td>3</td><td>05.02.2021</td></tr>
    <tr>
        <td>4</td><td>05.02.2021</td></tr>
    <tr>
        <td>2</td><td>06.02.2021</td></tr>
</Table>

### Задание
Используя библиотеку SQLAlchemy и программное обеспечение Юпитер Ноутбук создать структуру  (не наполняя ее данными) реляционной базы данных студентов "Students.db", содержащую таблицы: Students, Lecturer, Subject, University, Exam_marks. Применить необходимве ограничения. В качестве источника информации о конкретной структуре таблиц использовать учебную базу данных из методического пособия: _Астахова И.Ф, Толстобров А.П., Мельников В.М. SQL в примерах и задачах._

### Решение

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

from sqlalchemy import (MetaData, Table, Column, Integer, Numeric, String, DateTime, 
                        Boolean, ForeignKey, create_engine, PrimaryKeyConstraint, 
                        UniqueConstraint, CheckConstraint, Index, insert, BigInteger)

In [2]:
metadata = MetaData()
engine = create_engine('sqlite:///Students.db')

In [3]:
student=Table('student',metadata, 
              Column('student_id',Integer(),primary_key=True), 
              Column('surname',String(255),nullable=False), 
              Column('name',String(255),nullable=False), 
              Column('stipend',Integer(),nullable=False), 
              Column('kurs',Integer(),nullable=False), 
              Column('city',String(300)), 
              Column('birthday',DateTime(),default=dt.date.today), 
              Column('univ_id',Integer(),ForeignKey('university.univ_id'),nullable=False), 
              extend_existing=True)

In [4]:
subject=Table('subject',metadata, 
              Column('subj_id',Integer(),primary_key=True), 
              Column('subj_name',String(64),nullable=False, index=True), 
              Column('hour',Integer(),nullable=False, index=True), 
              Column('semester',Integer(),nullable=False), 
              extend_existing=True)

In [5]:
exam_marks=Table("exam_marks", metadata,
                Column("exam_id", Integer(), primary_key=True),
                Column("student_id", ForeignKey('student.student_id')), 
                Column("subj_id", ForeignKey('subject.subj_id')),
                Column("mark", Integer()), 
                Column("exam_date", DateTime(), nullable=False),
                extend_existing=True)

In [6]:
lecturer=Table('lecturer', metadata,
                Column('lecturer_id',Integer(),primary_key=True),
                Column('surname',String(30),nullable=False),
                Column('name',String(30),nullable=False),
                Column('city',String(30)),
                Column('univ_id',Integer(),ForeignKey('university.univ_id'),nullable=False))

In [7]:
subj_lect=Table('subj_lect',metadata,
        Column('lecturer_id',Integer(),ForeignKey('lecturer.lecturer_id')),
        Column('subj_id',Integer(),ForeignKey('subject.subj_id')),
        PrimaryKeyConstraint('lecturer_id','subj_id', name='user_subj'),
        extend_existing=True)

In [8]:
university=Table('university',metadata,
                Column('univ_id',Integer(),primary_key=True),
                Column('univ_name',String(255)),
                Column('rating',Integer()),
                Column('city',Integer()),
                extend_existing=True)

In [9]:
metadata.create_all(engine)

## Контрольная работа № 3

<Table>
    <tr>
        <th>Подгруппа</th><th>Дата</th></tr>
    <tr>
        <td>3</td><td>19.02.2021</td></tr>
    <tr>
        <td>2</td><td>20.02.2021</td></tr>
    <tr>
        <td>4</td><td>26.02.2021</td></tr>
</Table>


### Задание
Используя библиотеку SQLAlchemy и программное обеспечение Юпитер Ноутбук, наполните базу данных _Students.db_ данными из учебной базы данных студентов, описание которой содержится в учебном пособии: _Астахова И.Ф, Толстобров А.П., Мельников В.М. SQL в примерах и задачах_.

### Решение

В представленном решении для разных таблиц использовались разные способы: 
- одиночная вставка;
- вставка в цикле из списвка словарей, сформированного на основе данных таблицы;
- вставка в цикле из кортежей, сформированных "вручную".

Использование разных способов преследует демонстрационные цели. Студент в ходе контрольной работы может выбирать любой из представленных способов, либо предложить свой.

In [10]:
connection=engine.connect()

1. Одиночная вставка<br>
1.1. _insert_ как метод

In [11]:
ins=exam_marks.insert().values(
    exam_id=145,
    student_id=12,
    subj_id=10,
    mark=5,
    exam_date=dt.date(2006,1,12))
print(str(ins))
print(ins.compile().params)

INSERT INTO exam_marks (exam_id, student_id, subj_id, mark, exam_date) VALUES (:exam_id, :student_id, :subj_id, :mark, :exam_date)
{'exam_id': 145, 'student_id': 12, 'subj_id': 10, 'mark': 5, 'exam_date': datetime.date(2006, 1, 12)}


In [12]:
result=connection.execute(ins)
result.inserted_primary_key

[145]

In [13]:
ins=exam_marks.insert()

result=connection.execute(
    ins,
    exam_id=238,
    student_id=12,
    subj_id=10,
    mark=5,
    exam_date=dt.date(1999,6,17))

result.inserted_primary_key

[238]

In [14]:
ins=exam_marks.insert().values(
    (34,32,10,4,dt.datetime.strptime("23.01.2000","%d.%m.%Y")))

result=connection.execute(ins)
result.inserted_primary_key

[34]

1.2. _insert_ как функция

In [15]:
ins=insert(exam_marks).values(
    exam_id=639,
    student_id=55,
    subj_id=10,
    mark=None,
    exam_date=dt.date(1999,6,22)
)
result=connection.execute(ins)
result.inserted_primary_key

[639]

In [16]:
ins=insert(exam_marks).values(
    (43,6,22,4,dt.datetime.strptime("18.01.2000","%d.%m.%Y")))

result=connection.execute(ins)
result.inserted_primary_key

[43]

In [17]:
ins=insert(exam_marks).values(
    (75,55,10,5,dt.datetime.strptime("05.01.2000","%d.%m.%Y")))

result=connection.execute(ins)
result.inserted_primary_key

[75]

2. Множественная вставка<br>
2.1. Вставка из датафрейма

In [18]:
st=pd.read_csv('./Data/BD_Primer_Student.csv',sep=';',encoding='cp1251')
st

,STUDENT_ID,SURNAME,NAME,STIPEND,KURS,CITY,BIRTHDAY,UNIV_ID
0,1,Иванов,Иван,150,1,Орел,1982-12-03,10
1,3,Петров,Петр,200,3,Курск,1980-12-01,10
2,6,Сидоров,Вадим,150,4,Москва,1979-06-07,22
3,10,Кузнецов,Борис,0,2,Брянск,1981-12-08,10
4,12,Зайцева,Ольга,250,2,Липецк,1981-05-01,10
5,32,Котов,Павел,150,5,Белгород,NaN,14
6,55,Белкин,Вадим,250,5,Воронеж,1980-01-07,10
7,265,Павлов,Андрей,0,3,Воронеж,1979-11-05,10
8,276,Петров,Антон,200,4,NaN,1981-08-05,22
9,654,Лукин,Артем,200,3,Воронеж,1981-12-01,10


In [19]:
st.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STUDENT_ID  10 non-null     int64 
 1   SURNAME     10 non-null     object
 2   NAME        10 non-null     object
 3   STIPEND     10 non-null     int64 
 4   KURS        10 non-null     int64 
 5   CITY        9 non-null      object
 6   BIRTHDAY    9 non-null      object
 7   UNIV_ID     10 non-null     int64 
dtypes: int64(4), object(4)
memory usage: 768.0+ bytes


In [20]:
st.values

array([[1, 'Иванов', 'Иван', 150, 1, 'Орел', '1982-12-03', 10],
       [3, 'Петров', 'Петр', 200, 3, 'Курск', '1980-12-01', 10],
       [6, 'Сидоров', 'Вадим', 150, 4, 'Москва', '1979-06-07', 22],
       [10, 'Кузнецов', 'Борис', 0, 2, 'Брянск', '1981-12-08', 10],
       [12, 'Зайцева', 'Ольга', 250, 2, 'Липецк', '1981-05-01', 10],
       [32, 'Котов', 'Павел', 150, 5, 'Белгород', nan, 14],
       [55, 'Белкин', 'Вадим', 250, 5, 'Воронеж', '1980-01-07', 10],
       [265, 'Павлов', 'Андрей', 0, 3, 'Воронеж', '1979-11-05', 10],
       [276, 'Петров', 'Антон', 200, 4, nan, '1981-08-05', 22],
       [654, 'Лукин', 'Артем', 200, 3, 'Воронеж', '1981-12-01', 10]],
      dtype=object)

In [21]:
st_list=[]
for row in st.values:
    tmp={'student_id':row[0],
        'surname':row[1],
        'name':row[2],
        'stipend':row[3],
        'kurs':row[4],
        'city':row[5],
        'birthday': dt.datetime.strptime(row[6],'%Y-%m-%d') if type(row[6])==str else dt.date.today(),
        'univ_id':row[7]}
    st_list.append(tmp)
st_list

[{'student_id': 1,
  'surname': 'Иванов',
  'name': 'Иван',
  'stipend': 150,
  'kurs': 1,
  'city': 'Орел',
  'birthday': datetime.datetime(1982, 12, 3, 0, 0),
  'univ_id': 10},
 {'student_id': 3,
  'surname': 'Петров',
  'name': 'Петр',
  'stipend': 200,
  'kurs': 3,
  'city': 'Курск',
  'birthday': datetime.datetime(1980, 12, 1, 0, 0),
  'univ_id': 10},
 {'student_id': 6,
  'surname': 'Сидоров',
  'name': 'Вадим',
  'stipend': 150,
  'kurs': 4,
  'city': 'Москва',
  'birthday': datetime.datetime(1979, 6, 7, 0, 0),
  'univ_id': 22},
 {'student_id': 10,
  'surname': 'Кузнецов',
  'name': 'Борис',
  'stipend': 0,
  'kurs': 2,
  'city': 'Брянск',
  'birthday': datetime.datetime(1981, 12, 8, 0, 0),
  'univ_id': 10},
 {'student_id': 12,
  'surname': 'Зайцева',
  'name': 'Ольга',
  'stipend': 250,
  'kurs': 2,
  'city': 'Липецк',
  'birthday': datetime.datetime(1981, 5, 1, 0, 0),
  'univ_id': 10},
 {'student_id': 32,
  'surname': 'Котов',
  'name': 'Павел',
  'stipend': 150,
  'kurs': 5,
 

In [22]:
ins=student.insert()
result=connection.execute(ins, st_list)

In [23]:
lec=pd.read_csv('./Data/BD_primer_Lecturer.csv',sep=';',encoding='cp1251')
lec

,LECTURER_ID,SURNAME,NAME,CITY,UNIV_ID
0,24,Колесников,Борис,Воронеж,10
1,46,Никонов,Иван,Воронеж,10
2,74,Лагутин,Павел,Москва,22
3,108,Струков,Николай,Москва,22
4,276,Николаев,Виктор,Воронеж,10
5,328,Сорокин,Андрей,Орел,10


In [24]:
lec_list=[]
for item in lec.values:
    tmp={'lecturer_id':item[0],
        'surname':item[1],
        'name':item[2],
        'city':item[3],
        'univ_id':item[4]}
    lec_list.append(tmp)

In [25]:
ins=lecturer.insert()
result=connection.execute(ins, lec_list)

2.2. Вставка из кортежей, сформированных "вручную".

In [26]:
subject_list = [
    (10, 'Информатика', 56, 1),
    (22, 'Физика', 34, 1),
    (43, 'Математика', 56, 2),
    (56, 'История', 34, 4),
    (73, 'Физкультура', 34, 5),
    (94, 'Английский', 56, 3)
]

university_list = [
    (10, 'ВГУ', 296, 'Воронеж'),
    (11, 'НГУ', 345, 'Новосибирск'),
    (14, 'БГУ', 326, 'Белгород'),
    (15, 'ТГУ', 368, 'Томск'),
    (18, 'ВГМА', 327, 'Воронеж'),
    (22, 'МГУ', 606, 'Москва'),
    (32, 'РГУ', 416, 'Ростов')
]



In [27]:
def insert_values(table, values_list):
    for value in values_list:
        ins = table.insert().values(value)
        connection.execute(ins)

In [28]:
insert_values(university, university_list)
insert_values(subject, subject_list)